In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

quoting = 3 ignora tutte i doppi apici " nel testo. Evita errori in runtime perchè i metodi potrebbero confonderle con delle stringhe ecc.

In [62]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

Il dataset deve essere semplificato e ridotto eliminando parole inutili, verbi in tempi diversi ecc.

Le "stopwords" sono tutte le congiunzioni, articoli ecc. che non danno significato in piu alla frase.

PorterStemmer permette di individuare la parte importante di una parola, come ad esempio la parte al presente del verbo coniugato. (Es: love e loved dovranno occupare la stessa casella nella bag of words)

Le stopwords includono anche "not", che però non deve essere tolto dalle parole perchè è importante, in quanto cambia completamente il senso della frase.

In [63]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
for i in range(0, 1000):
  review = re.sub('[^a-zA-Z]', ' ', dataset["Review"][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words("english")
  all_stopwords.remove("not")
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = " ".join(review)
  corpus.append(review)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creare il modello bag of words significa creare una matrice sparsa in cui le righe sono le varie frasi, e nelle colonne sono contate le parole che appaiono nella frase. Ogni casella corrisponde ad una parola del dizionario. Il processo si chiama "tokenization".

E' consigliato inserire il numero massimo di parole contenute nella matrice. Per farlo è meglio far girare il programma una volta, verificare quante parole sono presenti, e poi diminuire il numero massimo in funzione di quante parole "rare" o poco rilevanti sono presenti (es: nomi di persona)

Ci sono 1566 parole, per cui è fattibile mettere come limite massimo 1500.

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values
print(len(X[0]))

1500


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Per esperienza, l'algoritmo Naive Bayes funziona bene per il NLP

In [66]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB(priors=None, var_smoothing=1e-09)
classifier.fit(X_train, y_train)

GaussianNB()

In [67]:
y_pred = classifier.predict(X_test)
delta = abs(y_pred-y_test)
# print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1), delta.reshape(len(delta),1) ), axis=1))

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 67  50]
 [ 20 113]]


0.72

Per fare la previsione di una singola recensione è necessario fare tutto il pre processing effettuato per l'intero dataset, applicato però solo alla singola frase.

In [69]:
new_review = 'I love this restaurant so much'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

new_review = 'I hate this restaurant so much'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

[1]
[0]
